In [1]:
!pip install -q -e ../.

In [2]:
import pandas as pd
import openai
from looptune import df_to_ds, balance_dataset,split_ds, looped_prompt_classification, load_PolarIs, get_response

In [58]:
# Local (host via LM studio, default url and api)
client = openai.OpenAI(base_url='http://localhost:1234/v1',  api_key='lm-studio')
model = 'model-local' #model name will be extracted via api, as there will be only one, if multimodel sessions are used, pass proper model name (multimodel is untested)

#Openai
# client = openai.OpenAI(base_url='https://api.openai.com/v1/',  api_key=input("provide OpenAI api key"))
# model = 'gpt-4o-2024-05-13'

def create_messages(text='_'):
    return [
        {
            'role': 'system',
             'content': """You are an analytical tool designed to identify emotions in texts. Your task is to analyze text provided for emotions. The only responses to consider are: POSITIVE, NEGATIVE, NEUTRAL.
             
             Do not infer emotions not explicitly stated in the text, and do not identify emotion types unspecified."""
        },
        {
            "role": "assistant",
            "content": "Based on the analysis, can uyou find emotions in the text? If so, identify only one of: POSITIVE, NEGATIVE, NEUTRAL. Do not identify emotions not listed."
        },
        {
            "role": "user",
            "content": f"""Analyze the following text for emotions: {text}"""
        },
    ]

par = {'temperature': 0.1,
       'max_tokens': 50,
       }

In [57]:
#get single response

response = get_response(create_messages("I am so happy to join your newest venture"), par, client, model)
response

In [3]:
# run in loop with 
df = pd.read_csv('example_data/SentimentAnalysisforFinancialNews.csv', encoding="ISO-8859-1", header=None)
df.columns = ['label', 'text']
df.head(5)


dataset, target_map = df_to_ds(df)
# dataset = split_ds(dataset, train_size=0.7)
dataset = balance_dataset(dataset)
dataset

          text
label         
negative   604
neutral   2879
positive  1363


Casting to class labels:   0%|          | 0/4846 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1812 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text'],
    num_rows: 1812
})

In [59]:


reversed_target_map = {v:k for k,v in target_map.items()}

val_txts, val_labels = dataset['text'], dataset['label']

looped_prompt_classification(val_txts[:100], 
              val_labels[:100], 
              model, 
              par, 
              client, 
              create_messages, 
              # output_path='reponses.csv', # if empty or doesn't end with '.csv' (for example dir path), basic filename will be created
              # prompt_log='prompt_log.csv') # if empty or doesn't end with '.csv' (for example dir path), basic filename will be created
                            )